In [1]:
import glob
import random
import itertools
import math
import multiprocessing
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from python_speech_features import mfcc, fbank, logfbank
from pydub import AudioSegment

In [2]:
reciters_paths = glob.glob('./reciters/*/*')

In [3]:
train_paths_dictionary = {}
test_paths_dictionary = {}

for i, j in itertools.groupby(reciters_paths, key=lambda x: x.split('/')[-2]):
    files = list(j)
    train_paths, test_paths = train_test_split(files, test_size=0.30)
    train_paths_dictionary[int(i)-1] = train_paths
    test_paths_dictionary[int(i)-1] = test_paths

In [4]:
# %%time
# training_data = {}
# for reciter_id, reciter_path in train_paths_dictionary.items():
#     training_data[reciter_id] = []
#     for clip_path in random.sample(reciter_path, min(1, len(reciter_path))):
#         clip = AudioSegment.from_mp3(clip_path).split_to_mono()[0]
#         clip_sample_rate = clip.frame_rate
#         nfft = 2 ** math.ceil(np.log2(0.025 * clip_sample_rate))
#         clip_sliced = list(clip[::6400])
#         print(reciter_id)
#         print(clip.channels)
#         print(clip_sample_rate)
#         samples = random.sample(clip_sliced, min(10, len(clip_sliced)))
#         for sample in samples:
#             features = mfcc(np.array(sample.get_array_of_samples()), samplerate=clip_sample_rate, nfft=nfft)
#             training_data[reciter_id].append(features)

In [5]:
%%time
def get_data_and_labels(paths_dictionary):
    number_of_labels = len(paths_dictionary)
    number_of_segments = 254
    number_of_features = 13
    x = []
    y = []
    for reciter_id, reciter_path in tqdm(paths_dictionary.items()):
        for clip_path in random.sample(reciter_path, min(10, len(reciter_path))):
            clip = AudioSegment.from_mp3(clip_path).split_to_mono()[0]
            clip_sample_rate = clip.frame_rate
            nfft = 2 ** math.ceil(np.log2(0.025 * clip_sample_rate))
            clip_sliced = list(clip[::2560])[:-1] # to avoid the last incomplete slice
    #         print(reciter_id)
    #         print(clip.channels)
    #         print(clip_sample_rate)
            samples = random.sample(clip_sliced, min(10, len(clip_sliced)))
            for sample in samples:
                features = mfcc(np.array(sample.get_array_of_samples()), samplerate=clip_sample_rate, nfft=nfft)
                x.append(features[:number_of_segments, :])
                y.append(reciter_id)
    x = np.array(x)
    y = np.array(y)
    
    return x, y

x_train, y_train = get_data_and_labels(train_paths_dictionary)
x_test, y_test = get_data_and_labels(test_paths_dictionary)

100%|██████████| 35/35 [11:08<00:00, 19.11s/it]

CPU times: user 25min 6s, sys: 2min 35s, total: 27min 42s
Wall time: 24min 15s


In [15]:
x_train_ = x_train.reshape((x_train.shape[0], x_train.shape[1] * x_train.shape[2]))
x_test_ = x_test.reshape((x_test.shape[0], x_test.shape[1] * x_test.shape[2]))

In [16]:
# from sklearn.linear_model import LogisticRegression

# clf = LogisticRegression()
# clf.fit(x_train_, y_train)
# pred = clf.predict(x_test_)
# accuracy_score(y_test, pred)

TypeError: predict() takes 2 positional arguments but 3 were given

In [ ]:
from keras.models import Model, Sequential, load_model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.layers import Input, Dense, LSTM, Dropout, Embedding, Bidirectional, SpatialDropout1D, Flatten, MaxPool1D
from keras.layers import Concatenate, Average, Add, GlobalAveragePooling1D, GlobalMaxPooling1D, CuDNNLSTM, CuDNNGRU, GRU
from keras.layers import BatchNormalization, GaussianNoise, GaussianDropout, AlphaDropout
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras import backend as K

In [ ]:
y_train_ = to_categorical(y_train)
y_test_ = to_categorical(y_test)

In [ ]:
class precision_callback(Callback):
    def __init__(self):
        super(precision_callback, self).__init__()

    def on_train_begin(self, logs={}):
        if not ('val_precision' in self.params['metrics']):
            self.params['metrics'].append('val_precision')

    def on_epoch_end(self, epoch, logs={}):
        logs['val_precision'] = float('-inf')
        if(self.validation_data):
            y_pred = self.model.predict(self.validation_data[0], batch_size = self.params['batch_size'])
            y_pred = np.array([np.argmax(p) for p in y_pred])
            
            y_test = self.validation_data[1]
            y_test = np.array([np.argmax(p) for p in y_test])
            
            logs['val_precision'] = precision_score(y_test, y_pred, average='micro')

In [ ]:
inp1 = Input(shape=(254, 13, ))
x = Flatten()(inp1)
# x = SpatialDropout1D(0.1)(x)
x = Dropout(0.2)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(35, activation="softmax")(x)

model = Model(inputs=inp1, outputs=x)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.fit(x_train, to_categorical(y_train), epochs=3, batch_size=1024, verbose=1, 
          validation_data=(x_test, to_categorical(y_test)), callbacks=[])

In [ ]:
sound = AudioSegment.from_mp3(path)

In [ ]:
rate = sound.frame_rate
signal = np.array(sound.get_array_of_samples())

In [ ]:
list(sound[::5000])

In [ ]:
for chunk in sound[::5000]:
    print(type(chunk))
#     mfcc(np.array(chunk.get_array_of_samples()))